In [30]:
import pandas as pd
import numpy as np
import nltk
df = pd.read_csv('../data/press_all.csv')

## Pre processing

In [31]:
# Replace nan with ""
df = df.fillna("")
df

,id,siren,url_article,title,text
0,0,572067684,http://www.autoactu.com/?page=2&id_actu=24240&...,autoactu.com - l'information automobile profes...,Bosch recrute des diplômés hautement qualifi...
1,1,790503973,http://www.ouest-france.fr/bretagne/lannion-22...,You have been blocked,Please enable JS and disable any ad blocker
2,2,433977980,http://www.businesswire.fr/news/fr/20160308006...,Axway s'associe à Navitas pour accompagner la ...,
3,3,495232373,http://www.next-finance.net/Sofimac-Partners-a...,Marchés Privés - Sofimac Partners apporte 450 ...,Présent au capital de cette jeune entreprise ...
4,4,572050169,http://rss.usinenouvelle.com/~r/a-la-une/~3/Gs...,"Les finances dans le vert, Latécoère veut mode...",\n\n\n\n \n\nIdentifiez-vous\n L'Usine Aéro P...
...,...,...,...,...,...
76509,76509,510050461,https://www.lejsl.com/edition-bresse/2019/10/1...,Sornay | Un salarié prend la succession de Lec...,JOURNAL DE SAÔNE ET LOIRE Conditions généra...
76510,76510,445720717,https://abonne.lunion.fr/id102153/article/2019...,,
76511,76511,480012277,https://www.lemondedudroit.fr/deals/66470-jean...,Jeantet conseille Formel D - LE MONDE DU DROIT...,UTILISATION DES COOKIES : en poursuivant votr...
76512,76512,414819409,http://www.leparisien.fr/val-de-marne-94/ile-d...,"Ile-de-France : à Villeneuve-Saint-Georges, le...","« S'ils veulent nous virer, qu'ils le fassent..."


In [32]:
#Delete all articles with length 0 
# Get names of articles with length 0
indexNames = df[ (df['text']) == "" ].index

# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)
df

,id,siren,url_article,title,text
0,0,572067684,http://www.autoactu.com/?page=2&id_actu=24240&...,autoactu.com - l'information automobile profes...,Bosch recrute des diplômés hautement qualifi...
1,1,790503973,http://www.ouest-france.fr/bretagne/lannion-22...,You have been blocked,Please enable JS and disable any ad blocker
3,3,495232373,http://www.next-finance.net/Sofimac-Partners-a...,Marchés Privés - Sofimac Partners apporte 450 ...,Présent au capital de cette jeune entreprise ...
4,4,572050169,http://rss.usinenouvelle.com/~r/a-la-une/~3/Gs...,"Les finances dans le vert, Latécoère veut mode...",\n\n\n\n \n\nIdentifiez-vous\n L'Usine Aéro P...
5,5,794458075,http://feedproxy.google.com/~r/TracesEcrites/~...,La jeune pousse In’Air Solutions traque les po...,MENU Bourgogne-Franche-Comté Grand-Est - ...
...,...,...,...,...,...
76508,76508,493255848,http://www.actu-transport-logistique.fr//dossi...,,Transport Supply Chain Magazine | Dossier...
76509,76509,510050461,https://www.lejsl.com/edition-bresse/2019/10/1...,Sornay | Un salarié prend la succession de Lec...,JOURNAL DE SAÔNE ET LOIRE Conditions généra...
76511,76511,480012277,https://www.lemondedudroit.fr/deals/66470-jean...,Jeantet conseille Formel D - LE MONDE DU DROIT...,UTILISATION DES COOKIES : en poursuivant votr...
76512,76512,414819409,http://www.leparisien.fr/val-de-marne-94/ile-d...,"Ile-de-France : à Villeneuve-Saint-Georges, le...","« S'ils veulent nous virer, qu'ils le fassent..."


In [33]:
import re
import contractions
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
from nltk.stem.snowball import FrenchStemmer
import spacy
from spacy_lefff import LefffLemmatizer, POSTagger

nlp = spacy.load('fr')
nlp.max_length = 68111459
french_lemmatizer = LefffLemmatizer()
nlp.add_pipe(french_lemmatizer, name='lefff')
translator = str.maketrans('', '', string.punctuation)


2020-03-25 16:33:02,167 - spacy_lefff.lefff - INFO - New LefffLemmatizer instantiated.
2020-03-25 16:33:02,168 - spacy_lefff.lefff - INFO - Reading lefff data...
2020-03-25 16:33:02,698 - spacy_lefff.lefff - INFO - Successfully loaded lefff lemmatizer


In [38]:


# Tokenize text
def preprocessing(sent):
        result = []
        sent = sent.replace('\n', ' ')
        # To lowercase 
        sent = sent.lower()
        # Remove numbers
        sent = re.sub("\d+", "", sent)
        # Delete punctuation 
        translator = str.maketrans('', '', string.punctuation)
        sent = sent.translate(translator)
        # Stemming words ex: books — book, looked — look
        # Lemmentization (same as stemming but based on knowledge base)
        doc = nlp(sent,disable = ['ner', 'parser'])
        for d in doc:
            result.append(d.lemma_)
        return result

def test(dataframe,index=0):
    f = open('data' + str(index) + '.txt' , 'w')
    for i,row in dataframe.iterrows():
        result =" ".join(preprocessing((row['text']) ))
        f.write( result + '\n')  # python will convert \n to os.linesep
    f.close()  

In [39]:
%%time
import multiprocessing
chunks = np.arra\


y_split(df,4)
manager = multiprocessing.Manager()
threads = []
for index,chunk in enumerate(chunks):
    thread = multiprocessing.Process(target=test, args=(chunk,index))
    thread.start()
    threads.append(thread)
for thread in threads:
    thread.join()


CPU times: user 81.4 ms, sys: 405 ms, total: 486 ms
Wall time: 25min 56s


In [40]:
! cat data*.txt > merge

In [41]:
! rm data*.txt

In [37]:
! cp merge prepro_with_punct

In [ ]:
# Check average length of article 
df['length'] = df.text.apply(lambda x: len(x))
max_ = df.length.max()
print("Max: {}".format(max_))
min_ = df.length.min()
print("Min: {}".format(min_))
mean = df.length.mean()
print("Mean: {}".format(mean))


In [14]:
f = open('data.txt', 'w')
for i,row in df.iterrows():
    f.write(row['text'] + '\n')  # python will convert \n to os.linesep
f.close()  

In [42]:
import fasttext
model = fasttext.train_unsupervised('merge',thread=4, minCount=10, dim=300)

In [43]:
import pandas as pd
df_mapping = pd.read_csv('../data/siren_name_map.csv')

In [45]:
from collections import defaultdict
mapping = defaultdict(str)
for i,row in df_mapping.iterrows():
    mapping[row.siren] = row.legal_name

In [46]:
mapping

defaultdict(str,
            {419838529: 'IPSEN',
             813883964: 'PEPTIMIMESIS',
             572060333: 'MERSEN',
             542104245: 'BANQUE PALATINE',
             399258755: 'SPIE OPERATIONS',
             542107651: 'ENGIE',
             552030967: 'VALEO',
             392032934: 'MSD VACCINS',
             328479753: 'IDI',
             799730759: 'GROUPE WINNCARE',
             332822485: 'HAULOTTE GROUP',
             351571757: 'GL EVENTS',
             481480465: 'SAFT GROUPE SA',
             317540581: 'TRANSGENE',
             395030844: 'SANOFI',
             652014051: 'CARREFOUR',
             342376332: 'ILIAD',
             390474898: 'CHARGEURS',
             310879499: 'BOURBON CORPORATION',
             702012956: 'ALTRAN TECHNOLOGIES',
             428783872: 'CREDIXIA',
             562018002: 'GAUMONT',
             352045454: "FINANCIERE DE L'ECHIQUIER",
             420713935: 'ERAAM',
             811849595: 'IENA VENTURE',
             48077232

In [47]:
#df_nearest = pd.DataFrame(columns=['siren', 'legal_name', 'nearest_neighbours'])
# Append rows in Empty Dataframe by adding dictionaries
#dfObj = dfObj
nearest = defaultdict(str)
for siren,name in mapping.items():
    nearest[name] = model.get_nearest_neighbors(name)

In [48]:
nearest

defaultdict(str,
            {'IPSEN': [(0.6497787833213806, 'gmt'),
              (0.5332005620002747, 'quartz'),
              (0.523452877998352, 'rviewerpreference'),
              (0.5216014385223389, 'producer'),
              (0.5119101405143738, 'typ'),
              (0.5087929368019104, 'count'),
              (0.5073139667510986, 'thalos'),
              (0.5025973916053772, 'metadata'),
              (0.5017903447151184, 'onxvkksymbol'),
              (0.49781906604766846, 'pdfcontext')],
             'PEPTIMIMESIS': [(0.4627830386161804, '͆'),
              (0.4527222216129303, 'bpf'),
              (0.4516712725162506, 'xdgo'),
              (0.44939494132995605, 'vј'),
              (0.4489171802997589, 'iө'),
              (0.4434405565261841, '̱j\x7f'),
              (0.4426780939102173, '藈'),
              (0.4422869384288788, 'աt'),
              (0.43944987654685974, 'ꝯ'),
              (0.43870019912719727, 'womoc')],
             'MERSEN': [(0.6346310973167419, 'bo

In [16]:
df_mapping

,siren,legal_name
0,419838529,IPSEN
1,813883964,PEPTIMIMESIS
2,572060333,MERSEN
3,542104245,BANQUE PALATINE
4,399258755,SPIE OPERATIONS
...,...,...
30173,834289373,LAGARDERE MEDIA NEWS
30174,813956265,E-VALLEY SERVICES 2
30175,440088938,SENVION FRANCE
30176,510050461,LECUELLE ELECTRICITE


In [17]:
df_nearest = pd.DataFrame(columns=['siren', 'legal_name', 'nearest_neighbours'])
df_nearest.append({'siren': 'test', 'legal_name': 'test', 'nearest_neighbours': model.get_nearest_neighbors(name,5)}, ignore_index=True)

,siren,legal_name,nearest_neighbours
0,test,test,"[(0.7487287521362305, ë|), (0.7282547354698181..."
